In [5]:
import torch
device = torch.device('cuda')
torch.cuda.is_available()

True

In [2]:
from kp_api import KeypointExtractor

In [7]:
kp_cfg_path='configs/wholebody_2d_keypoint/rtmpose/cocktail14/rtmw-l_8xb320-270e_cocktail14-384x288.py'
kp_ckpt_path='rtmw-dw-x-l_simcc-cocktail14_270e-384x288-20231122.pth'

In [8]:
kp_extractor=KeypointExtractor(kp_cfg_path, kp_ckpt_path, device=device)

Loads checkpoint by local backend from path: rtmw-dw-x-l_simcc-cocktail14_270e-384x288-20231122.pth


c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\site-packages\mmengine\runner\checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = tor

Keypoint extractor model loaded successfully.


In [18]:
import os
import sys
import cv2
import subprocess
import tempfile
import numpy as np
import pandas as pd

def download_and_extract_keypoints(url, youtube_id, label, start, end, kp_extractor, output_dir):
    start_sec, end_sec = float(start), float(end)
    os.makedirs(os.path.join(output_dir, label), exist_ok=True)

    with tempfile.TemporaryDirectory() as tmpdir:
        video_path = os.path.join(tmpdir, "temp_clip.mp4")

        # 🔹 Descargar video completo con yt-dlp
        cmd = [
            sys.executable, "-m", "yt_dlp",
            "-f", "mp4",
            "-o", video_path,
            url
        ]
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"⚠️ Error descargando {youtube_id}: {result.stderr.strip()}")
            return

        # 🔹 Recortar clip en memoria usando OpenCV
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        if start_sec >= duration:
            print(f"⚠️ {youtube_id}: start fuera de rango ({start_sec}s >= {duration}s)")
            cap.release()
            return

        start_frame = int(start_sec * fps)
        end_frame = int(min(end_sec, duration) * fps)

        frames = []
        for i in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break
            if start_frame <= i <= end_frame:
                frames.append(frame)
        cap.release()

        if not frames:
            print(f"⚠️ {youtube_id}: sin frames válidos entre {start}-{end}")
            return

        # 🔹 Guardar clip temporal
        trimmed_path = os.path.join(tmpdir, "trimmed.mp4")
        h, w, _ = frames[0].shape
        out = cv2.VideoWriter(trimmed_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        for f in frames:
            out.write(f)
        out.release()

        # 🔹 Extraer keypoints
        keypoints = kp_extractor.extract_from_video(trimmed_path)
        if keypoints is None:
            print(f"⚠️ No se detectaron keypoints en {youtube_id}")
            return

        # 🔹 Guardar keypoints como .npy
        save_path = os.path.join(output_dir, label, f"{youtube_id}.npy")
        np.save(save_path, keypoints)
        print(f"✅ Guardado {save_path}")

In [19]:
N = 2

# Leer anotaciones desde CSV
df = pd.read_csv("../data/train.csv")  # debe tener youtube_id, start_time, end_time, label

for idx, row in df.iterrows():
    if idx >= N:
        break
    youtube_id = row['youtube_id']
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = row['label']
    
    # Descargar clip usando yt-dlp (del segundo 'start' al 'end')
    url = f"https://youtu.be/{youtube_id}"
    out_dir = os.path.join("output_dir", label)
    download_and_extract_keypoints(url, youtube_id, label, start, end, kp_extractor, out_dir)

Extracting keypoints from 251 frames...


Processing Frames:   0%|          | 0/251 [00:00<?, ?it/s]c:\Users\48113164\Documents\GitHub\mmpose-prueba\mmpose\mmpose\models\utils\csp_layer.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
c:\Users\48113164\Documents\GitHub\mmpose-prueba\mmpose\mmpose\models\backbones\csp_darknet.py:118: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Processing Frames: 100%|██████████| 251/251 [00:18<00:00, 13.44it/s]


✅ Guardado output_dir\abseiling\abseiling\-3B32lodo2M.npy
Extracting keypoints from 300 frames...


Processing Frames: 100%|██████████| 300/300 [00:26<00:00, 11.15it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'output_dir\\abseiling\\abseiling\\-7kbO0v4hag.npy'

In [25]:
for idx, row in df.iterrows():
    if idx >= N:
        break
    youtube_id = row['youtube_id']
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = row['label']
    out_dir = os.path.join("output_dir", label)
    os.makedirs(out_dir, exist_ok=True)
    out_temporal_path = os.path.join(out_dir, f'{youtube_id}_{start}_{end}')
    out_path = os.path.join(out_dir, f"{youtube_id}_{start}_{end}_keypoints.npy")

    
    # Descargar clip usando yt-dlp (del segundo 'start' al 'end')
    url = f"https://youtu.be/{youtube_id}"
    # 4. Comando yt-dlp (MODIFICADO)
    cmd_yt = [
        'yt-dlp', '--quiet',
        # Agrega esto para saltar verificaciones de edad
        '--age-limit', '99', 
        # Formato mucho más simple y robusto.
        # Pide el mejor mp4, si no, el mejor video disponible.
        '-f', 'best[ext=mp4]/best', 
        url,
        '-o', '-' # Salida a stdout
    ]

    # 5. Comando ffmpeg:
    #    -i -  significa "leer video desde stdin"
    #    -ss [start] y -to [end] aplican el corte
    cmd_ff = [
        'ffmpeg',
        '-i', '-',  # ¡Clave! Entrada desde stdin
        '-ss', str(start),
        '-to', str(end),
        '-c:v', 'libx264', # Forzar re-codificación, es más robusto
        '-c:a', 'aac',
        out_temporal_path,
        '-y' # Sobrescribir archivo temporal
    ]

    # 6. Ejecutar ambos procesos y conectarlos
    p_yt = None
    p_ff = None
    try:
        # Iniciar yt-dlp, enviando su salida a la tubería
        p_yt = subprocess.Popen(cmd_yt, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8')
        
        # Iniciar ffmpeg, leyendo su entrada desde la tubería
        p_ff = subprocess.Popen(cmd_ff, stdin=p_yt.stdout, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8')
        
        # Cerrar el stdout de p_yt en este proceso
        # (Es necesario para que p_yt reciba un "Broken pipe" si p_ff termina)
        p_yt.stdout.close() 

        # Esperar a que ffmpeg termine y obtener sus salidas
        # (yt-dlp terminará solo cuando ffmpeg deje de leer)
        ff_stdout, ff_stderr = p_ff.communicate()

        if p_ff.returncode != 0:
            print(f"⚠️ Error en ffmpeg (video {youtube_id}): {ff_stderr.strip()}")
            # Capturar también error de yt-dlp por si acaso
            _, yt_stderr = p_yt.communicate()
            print(f"Info de yt-dlp: {yt_stderr.strip()}")
            continue

    except Exception as e:
        print(f"Error al ejecutar Popen: {e}")
        # Asegurarse de que los procesos mueran si algo falla
        if p_yt: p_yt.kill()
        if p_ff: p_ff.kill()
        continue

    # --- FIN DE LA SOLUCIÓN 'PIPE' ---

    # 7. Si todo fue bien, extraer keypoints (el resto de tu script)
    try:
        print(f"Extrayendo keypoints de {out_temporal_path}...")
        keypoints = kp_extractor.extract_from_video(out_temporal_path)
    
        # 8. Guardar los keypoints
        np.save(out_path, keypoints)
        print(f"✅ Keypoints guardados en: {out_path}")
        
    except Exception as e:
        print(f"🚨 Error extrayendo keypoints de {youtube_id}: {e}")
        
    finally:
        # 9. Eliminar el video temporal
        if os.path.exists(out_temporal_path):
            os.remove(out_temporal_path)

Exception in thread Thread-97:
Traceback (most recent call last):
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
ValueError: I/O operation on closed file.


⚠️ Error en ffmpeg (video -3B32lodo2M): ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cu

Exception in thread Thread-101:
Traceback (most recent call last):
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\48113164\AppData\Local\miniconda3\envs\torch-gpu\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
ValueError: I/O operation on closed file.


⚠️ Error en ffmpeg (video -7kbO0v4hag): ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cu

In [22]:
import pandas as pd
import subprocess
import os
import numpy as np

# Número de videos a procesar
N = 2

# Leer anotaciones desde CSV
df = pd.read_csv("../data/train.csv")  # debe tener youtube_id, start_time, end_time, label

for idx, row in df.iterrows():
    if idx >= N:
        break
    youtube_id = row['youtube_id']
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = row['label']
    out_dir = os.path.join("output_dir", label)
    os.makedirs(out_dir, exist_ok=True)
    out_temporal_path = os.path.join(out_dir, f'{youtube_id}_{start}_{end}')
    
    # Descargar clip usando yt-dlp (del segundo 'start' al 'end')
    url = f"https://youtu.be/{youtube_id}"
    cmd = [
        'yt-dlp', '--quiet', '--no-warnings',
        # Pasa los argumentos de corte directamente a ffmpeg
        '--external-downloader', 'ffmpeg',
        '--external-downloader-args', f"ffmpeg_i:-ss {start} -to {end}",
        '-o', out_temporal_path,
        '--recode-video', 'mp4',
        url
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)

    if result.returncode != 0:
        print(f"⚠️ Error descargando {youtube_id}: {result.stderr}")
        continue
    
    # Extraer keypoints del clip descargado
    keypoints = kp_extractor.extract_from_video("temp_clip.mp4")
    
    out_path = os.path.join(out_dir, f"{youtube_id}_{start}_{end}_keypoints.npy")
    np.save(out_path, keypoints)
    
    # Eliminar video temporal
    os.remove("temp_clip.mp4")


Deprecated Feature: Support for Python version 3.9 has been deprecated. Please update to Python 3.10 or above


ERROR: ffmpeg exited with code 1

⚠️ Error descargando -3B32lodo2M: Deprecated Feature: Support for Python version 3.9 has been deprecated. Please update to Python 3.10 or above


ERROR: ffmpeg exited with code 1


Deprecated Feature: Support for Python version 3.9 has been deprecated. Please update to Python 3.10 or above


ERROR: ffmpeg exited with code 1

⚠️ Error descargando -7kbO0v4hag: Deprecated Feature: Support for Python version 3.9 has been deprecated. Please update to Python 3.10 or above


ERROR: ffmpeg exited with code 1

